# **Dependincies**

In [1]:
%%capture
!pip install polars
!pip install lightgbm==4.2.0
!pip install catboost==1.2.7
!pip install xgboost==2.0.3
!pip install optuna
!pip install lifelines

In [2]:
# from pytorch_tabnet.metrics import Metric
# from pytorch_tabnet.tab_model import TabNetRegressor

In [41]:
import shutil
import pandas as pd
import numpy as np
import re
import gc
import os
import math
import importlib
import random
import warnings
import pickle
import joblib
from tqdm import tqdm
import sys

from scipy.stats import skew, kurtosis
from collections import Counter,defaultdict
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import lightgbm as lgb
import catboost as cat, gc
import xgboost as xgb

print(lgb.__version__)
print(cat.__version__)
print(xgb.__version__)

from lightgbm import LGBMRegressor, LGBMClassifier
from catboost import CatBoostRegressor, CatBoostClassifier, Pool
from sklearn.svm import SVR
from xgboost import XGBRegressor, XGBClassifier

import optuna

from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss


import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau, CosineAnnealingWarmRestarts
import torch.nn.functional as F

from transformers import AutoModel,AutoTokenizer

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')

warnings.simplefilter('ignore')
warnings.filterwarnings('ignore')
sys.path.append('/notebooks/CIBMTR/utils')
sys.path.append('/notebooks/CIBMTR/py')

4.2.0
1.2.7
2.0.3


In [49]:
import tuning
import utils as my_utils
from metric import score
from utils import clr
importlib.reload(my_utils)
importlib.reload(tuning)

4.2.0
1.2.7
2.0.3


<module 'tuning' from '/notebooks/CIBMTR/py/tuning.py'>

In [50]:
# class clr:
#     BLACK     = '\033[30m'
#     RED       = '\033[31m'
#     GREEN     = '\033[32m'
#     YELLOW    = '\033[33m'
#     BLUE      = '\033[34m'
#     PURPLE    = '\033[35m'
#     CYAN      = '\033[36m'
#     WHITE     = '\033[37m'
#     END       = '\033[0m'
#     BOLD      = '\033[1m'
#     UNDERLINE = '\033[4m'
#     INVISIBLE = '\033[08m'
#     REVERCE   = '\033[07m'
#     S = '\033[1m' + '\033[95m'
#     E = '\033[0m'

# my_colors = ["#761D80", "#9926A6", "#9C69C9",
#              "#6C91BF", "#58BCC6", "#4AD1B2",
#              "#4BF1B2"]

# CMAP1 = ListedColormap(my_colors)

# **Config**

In [51]:
train_path = '/notebooks/CIBMTR/data/train.csv'
train_dic_path = '/notebooks/CIBMTR/data/data_dictionary.csv'

test_path = '/notebooks/CIBMTR/data/test.csv'
sub_path = '/notebooks/CIBMTR/data/sample_submission.csv'

In [52]:
train = pd.read_csv(train_path)
# dictionary_tow = pd.read_csv(train_dic_path)
dictionary = pd.read_csv('/notebooks/CIBMTR/output/dictionary.csv', encoding='cp932').drop('Unnamed: 0', axis=1)
test = pd.read_csv(test_path)
submission = pd.read_csv(sub_path)

In [53]:
TARGETS = ['efs_time', 'efs']

# **UM FEATURE ENGINNERING**

## **FE FUNCTIONS**

In [54]:
data=train
importlib.reload(my_utils)

<module 'utils' from '/notebooks/CIBMTR/utils/utils.py'>

In [55]:
print(f'INITIAL DATA Shape:{data.shape}')
print('')
FE = my_utils.CIBMTR_FE(mode='train')
data, _ = FE.trans_obj_cate(data)

print('')
print(f'Final DATA Shape:{data.shape}')

INITIAL DATA Shape:(28800, 60)


Final DATA Shape:(28800, 60)


## **CONCAT OOF PREDICTIONS**

In [56]:
# ####
# # USING GDBT MODEL OOF
# ####
# EXP_NUM_LIST = ['EXP-39','EXP-40','EXP-43']
# predictions={}
# for EXP_NUM in EXP_NUM_LIST:
#     SAVE_PATH = f'{my_config.general.model_save_path}/TREE_{EXP_NUM}'
#     oof_preds = joblib.load(f'{SAVE_PATH}/oof_preds_{EXP_NUM}.pkl')
#     oof_preds = {key: np.clip(value, a_min=-1.0, a_max=1.0) for key, value in oof_preds.items()}
#     oof_preds = {model: oof_preds[model].flatten() for model in oof_preds}
    
#     cols = [f"{col.split('_')[0]}-{EXP_NUM}" for col in oof_preds]
#     oof_preds_df = pd.DataFrame(oof_preds)
#     oof_preds_df.columns = cols
#     predictions[EXP_NUM]=(oof_preds_df)

# predictions_df = pd.DataFrame()
# for col in predictions:
#     tmp_df = predictions[col]
#     predictions_df = pd.concat([predictions_df, tmp_df], axis=1)
# predictions_df = predictions_df.astype('float64')

# true = train['utility_agent1'].copy()
# print(len(predictions))

In [57]:
# ####
# # USING NN MODEL OOF
# ####

# # EXP_NUM_NN_LIST = 0
# # EXP_NUM_NN_LIST = ['EXP-50', 'EXP-68']
# EXP_NUM_NN_LIST = []
# if len(EXP_NUM_NN_LIST)!=0:
#     for EXP_NUM in EXP_NUM_NN_LIST:
#         SAVE_PATH = f'{my_config.general.model_save_path}/NN_{EXP_NUM}'
#         oof_preds = pd.read_csv(f'{SAVE_PATH}/oof.csv')
#         predictions_df[f'nn-{EXP_NUM}'] = oof_preds['pred']
# predictions_df.head(3)

# ####
# # USING DTNN MODEL OOF
# ####
# # EXP_NUM_NN_LIST = 0
# # EXP_NUM_NN_LIST = ['EXP-17']
# EXP_NUM_NN_LIST = []
# if len(EXP_NUM_NN_LIST)!=0:
#     for EXP_NUM in EXP_NUM_NN_LIST:
#         oof_preds = joblib.load(f'model/DTNN_{EXP_NUM}/oof_preds_{EXP_NUM}.pkl')
#         oof_preds = pd.DataFrame(oof_preds['dtnn'], columns=['pred'])
#         predictions_df[f'dtnn-{EXP_NUM}'] = oof_preds['pred']
# predictions_df.head(3)

In [58]:
# USE_OOF = False

# if USE_OOF:
#     data = pd.concat([data, predictions_df], axis=1)
#     # data = FE.fe6(data.copy())
#     # exp_num = 'EXP-35'
#     # data = FE.drop_cols_fi3(data.copy(), 
#     #                       save_path='model', 
#     #                       exp_num=f'TREE_{exp_num}', 
#     #                       topn=25)

# data.replace([np.inf, -np.inf], np.nan, inplace=True)

In [59]:
# exp_num = 'EXP-48'
# data = FE.drop_cols_fi3(data.copy(), 
#                       save_path='model', 
#                       exp_num=f'TREE_{exp_num}', 
#                       topn=30)

# **OOF**

In [60]:
N_SPLITS=5
# kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=42)
kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=1)
for i, (_, val_index) in enumerate(kf.split(data, data.race_group)):
    data.loc[val_index, "fold"] = i
data.head(3)

,ID,dri_score,psych_disturb,cyto_score,diabetes,hla_match_c_high,hla_high_res_8,tbi_status,arrhythmia,hla_low_res_6,...,donor_related,melphalan_dose,hla_low_res_8,cardiac,hla_match_drb1_high,pulm_moderate,hla_low_res_10,efs,efs_time,fold
0,0,N/A - non-malignant indication,No,NaN,No,NaN,NaN,No TBI,No,6.0,...,Unrelated,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,42.356,0.0
1,1,Intermediate,No,Intermediate,No,2.0,8.0,"TBI +- Other, >cGy",No,6.0,...,Related,"N/A, Mel not given",8.0,No,2.0,Yes,10.0,1.0,4.672,1.0
2,2,N/A - non-malignant indication,No,NaN,No,2.0,8.0,No TBI,No,6.0,...,Related,"N/A, Mel not given",8.0,No,2.0,No,10.0,0.0,19.793,1.0


## **CHECK DATA**

In [61]:
FEATURES = data.drop(TARGETS+['fold']+['ID'], axis=1).columns.tolist()
len(FEATURES)

57

In [62]:
categorical_cols = data.drop(columns=TARGETS + ['ID']).select_dtypes(include=['object', 'category']).columns.to_list()
numerical_cols = data.drop(columns=TARGETS + ['ID']).select_dtypes(include=['number']).columns.to_list()

print(clr.RED + f'categorical_cols : {len(categorical_cols)}')
print(clr.RED + f'num_cols : {len(numerical_cols)}')

categorical_cols : 35
num_cols : 23


In [63]:
data[categorical_cols] = data[categorical_cols].astype(str).astype('category')

In [64]:
data = FE.create_target4(data)

# **TUENING HYPERPARAMETER**

In [98]:
from scipy.stats import rankdata
import metric
from metric import score
importlib.reload(metric)
importlib.reload(tuning)

4.2.0
1.2.7
2.0.3


<module 'tuning' from '/notebooks/CIBMTR/py/tuning.py'>

In [99]:
# use_optuna = True

In [100]:
tuning.optimize_params(
                data=data,
                features=FEATURES,
                target_type='target4',
                n_split=N_SPLITS, 
                cat_cols=categorical_cols,
                cv_fold_func=np.average, 
                model_name='cat')

[I 2025-01-13 06:27:23,880] A new study created in memory with name: Optimize boosting hyperparameters


target4
0:	learn: -120317.5009902	test: -25868.5502842	best: -25868.5502842 (0)	total: 47.1ms	remaining: 4m 31s
500:	learn: -115525.2494858	test: -25256.7017910	best: -25254.5183981 (462)	total: 20.4s	remaining: 3m 34s
Stopped by overfitting detector  (400 iterations wait)

bestTest = -25254.5184
bestIteration = 462

Shrink model to first 463 iterations.
American Indian or Alaska Native
0.6842452538403758
Asian
0.7019189616234593
Black or African-American
0.6717925980596
More than one race
0.6886024118743957
Native Hawaiian or other Pacific Islander
0.6669903947849993
White
0.6414982294315927
fold0 C-index:0.65674639682316
target4
0:	learn: -120663.1250082	test: -25588.3746119	best: -25588.3746119 (0)	total: 44.1ms	remaining: 4m 14s
500:	learn: -115745.5129933	test: -25046.0323118	best: -25038.4219521 (412)	total: 18.3s	remaining: 3m 12s
Stopped by overfitting detector  (400 iterations wait)

bestTest = -25038.42195
bestIteration = 412

Shrink model to first 413 iterations.
American In

[W 2025-01-13 06:29:21,721] Trial 0 failed with parameters: {'n_estimators': 5765, 'learning_rate': 0.056591927557044425, 'max_depth': 7, 'l2_leaf_reg': 0.17219666765365393, 'min_child_samples': 9} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/notebooks/CIBMTR/py/tuning.py", line 164, in <lambda>
    study.optimize(lambda trial: objective(trial, data, features, target_type, n_split, cat_cols, cv_fold_func, model_name),
                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/notebooks/CIBMTR/py/tuning.py", line 119, in objective
    model.fit(X_train,
  File "/usr/local/lib/python3.11/dist-packages/catboost/core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embed

KeyboardInterrupt: 

In [33]:
# """ Optuna Objective """

#######確認する!!!!!!!!!!########
fold_imp_flag = False
###############################
def objective(trial, 
              data=data,
              features=FEATURES,
              targets=TARGETS,
              n_split=N_SPLITS, 
              cat_cols=categorical_cols,
              cv_fold_func=np.average, 
              model_name='cat'):
    # cat base
    if model_name=='cat':
        param = {
            'loss_function': 'Cox',
            'grow_policy': 'SymmetricTree',
            'random_seed' : 2025,
            'n_estimators': trial.suggest_int('n_estimators', 1000, 10000),
            'learning_rate': trial.suggest_float('learning_rate', 0.005, 0.09, log=True), 
            'max_depth': trial.suggest_int('max_depth', 4, 16),
            'verbose': 500,
            'task_type': 'CPU',
            'colsample_bylevel': trial.suggest_float('l2_leaf_reg', 0.05, 0.9, log=True),
            'min_child_samples': trial.suggest_int('min_child_samples', 1, 10),
            'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.05, 0.9, log=True),
            'subsample': trial.suggest_float('l2_leaf_reg', 0.01, 0.9, log=True),
        }
    
    """ Fold Iter """
    best_loss_score = []
    oof_preds = np.zeros(len(data))
    for fold in range(n_split):
        """ Dataset """
        X_train = data[data["fold"] != fold][features]
        Y_train = data[data["fold"] != fold]['target4']
        X_val = data[data["fold"] == fold][features]
        Y_val = data[data["fold"] == fold]['target4']
        val_idx = data[data["fold"] == fold].index
        
        """ Train Model """
        if model_name=='lgb':
            model = LGBMRegressor(**param, 
                                  categorical_feature=cat_cols)
            # model = LGBMRegressor(**param)
            model.fit(X_train,
                      Y_train,
                      eval_metric='rmse',
                      eval_set=[(X_val, Y_val)],
                      callbacks=[lgb.early_stopping(stopping_rounds=400, verbose=True),
                                 lgb.log_evaluation(period=400)], # Logの出力 100毎に出力
                     )
            
        elif model_name=='cat':
            model = CatBoostRegressor(**param, cat_features=cat_cols)
            model.fit(X_train,
                      Y_train,
                      eval_set=[(X_val, Y_val)],
                      early_stopping_rounds=400)
            
        elif model_name == 'xgb':
            model = XGBRegressor(**param, 
                                 enable_categorical=True,
                                 tree_method='gpu_hist')
            model.fit(X_train,
                      Y_train,
                      eval_set=[(X_train, Y_train), (X_val, Y_val)],
                      early_stopping_rounds=400,
                      verbose=400)
        """ Eval """
        preds = model.predict(X_val)
        oof_preds[val_idx] = preds
        
        y_true_fold = data.iloc[val_idx][['ID', 'efs', 'efs_time', 'race_group']].copy()
        y_pred_fold = data.iloc[val_idx][['ID']].copy()
        y_pred_fold['prediction'] = preds
        loss_val = score(y_true_fold, y_pred_fold, 'ID')
        best_loss_score.append(loss_val)
        print(clr.RED + f'fold{fold} C-index:{loss_val}' + clr.END)
        
        del X_train, X_val, Y_train, Y_val, model, preds
        gc.collect()

    # save_trial_params(trial.number, param, score)
    """ Finaly Logging """
    print('-'*70)
    print(f"-[INFO] Trial OOF(Log Loss): {np.mean(best_loss_score)}")
    print(f"-[INFO] Trial All Fold(Log Loss): {best_loss_score}")
    print('-'*70)
    
    
    return cv_fold_func(best_loss_score)

In [32]:
%%time
if use_optuna:
    study = optuna.create_study(direction='maxmize', study_name='Optimize boosting hyperparameters')
    study.optimize(objective, n_trials=100)

[I 2025-01-13 05:23:19,593] A new study created in memory with name: Optimize boosting hyperparameters


0:	learn: -120441.6261053	test: -25898.2071903	best: -25898.2071903 (0)	total: 39ms	remaining: 3m 56s
500:	learn: -118312.8539285	test: -25354.8654144	best: -25354.8654144 (500)	total: 21.3s	remaining: 3m 55s
1000:	learn: -117989.4666606	test: -25302.4997063	best: -25302.4997063 (1000)	total: 44.4s	remaining: 3m 44s
1500:	learn: -117755.6892004	test: -25277.0509374	best: -25277.0426953 (1499)	total: 1m 6s	remaining: 3m 20s
2000:	learn: -117552.1282941	test: -25262.2747866	best: -25262.2747866 (2000)	total: 1m 26s	remaining: 2m 54s
2500:	learn: -117382.8809147	test: -25250.8928079	best: -25250.8867742 (2499)	total: 1m 50s	remaining: 2m 36s
3000:	learn: -117236.5008885	test: -25244.0831870	best: -25244.0831870 (3000)	total: 2m 15s	remaining: 2m 17s
3500:	learn: -117107.0577205	test: -25239.0911221	best: -25239.0911221 (3500)	total: 2m 37s	remaining: 1m 54s
4000:	learn: -116989.4232604	test: -25237.0310601	best: -25237.0262993 (3990)	total: 3m 2s	remaining: 1m 33s
4500:	learn: -116874.864

[I 2025-01-13 05:46:53,892] Trial 0 finished with value: 0.6666534919422988 and parameters: {'n_estimators': 6050, 'learning_rate': 0.007752159793217138, 'max_depth': 5, 'l2_leaf_reg': 0.49065357762273537, 'min_child_samples': 9}. Best is trial 0 with value: 0.6666534919422988.


----------------------------------------------------------------------
-[INFO] Trial OOF(Log Loss): 0.6666534919422988
-[INFO] Trial All Fold(Log Loss): [0.6595427626598952, 0.6617132571077968, 0.6669859756858962, 0.6721617489380536, 0.6728637153198528]
----------------------------------------------------------------------
0:	learn: -120351.0080692	test: -25873.8065162	best: -25873.8065162 (0)	total: 24ms	remaining: 3m 57s
500:	learn: -117767.3202177	test: -25278.9704323	best: -25278.9704323 (500)	total: 12.2s	remaining: 3m 48s
1000:	learn: -117313.5703328	test: -25246.7960958	best: -25246.6559910 (996)	total: 25.1s	remaining: 3m 43s
1500:	learn: -116960.2117962	test: -25235.0287112	best: -25234.9663035 (1496)	total: 37.9s	remaining: 3m 32s
2000:	learn: -116656.5827200	test: -25232.5943452	best: -25230.8803727 (1864)	total: 49.2s	remaining: 3m 14s
2500:	learn: -116377.6930683	test: -25233.1646320	best: -25229.1479294 (2328)	total: 1m 1s	remaining: 3m 2s
Stopped by overfitting detector 

[I 2025-01-13 05:52:12,669] Trial 1 finished with value: 0.6658457681709964 and parameters: {'n_estimators': 9909, 'learning_rate': 0.06002766139978675, 'max_depth': 4, 'l2_leaf_reg': 0.060654852114078416, 'min_child_samples': 3}. Best is trial 1 with value: 0.6658457681709964.


----------------------------------------------------------------------
-[INFO] Trial OOF(Log Loss): 0.6658457681709964
-[INFO] Trial All Fold(Log Loss): [0.6613711859730522, 0.6610956622870998, 0.6632410445791581, 0.6723815219893015, 0.6711394260263703]
----------------------------------------------------------------------
0:	learn: -120445.1292638	test: -25898.9836348	best: -25898.9836348 (0)	total: 30.2ms	remaining: 3m 10s
500:	learn: -118447.7969649	test: -25386.4111369	best: -25386.4111369 (500)	total: 14.3s	remaining: 2m 45s
1000:	learn: -118133.7539744	test: -25326.7907575	best: -25326.7907575 (1000)	total: 29.5s	remaining: 2m 35s
1500:	learn: -117939.5689051	test: -25301.8992977	best: -25301.8992977 (1500)	total: 45.7s	remaining: 2m 26s
2000:	learn: -117772.1362985	test: -25285.4255146	best: -25285.4255146 (2000)	total: 1m 2s	remaining: 2m 13s
2500:	learn: -117630.3055951	test: -25274.8634999	best: -25274.8462298 (2499)	total: 1m 18s	remaining: 1m 58s
3000:	learn: -117505.132231

[W 2025-01-13 06:02:21,688] Trial 2 failed with parameters: {'n_estimators': 6301, 'learning_rate': 0.0064280816814082245, 'max_depth': 5, 'l2_leaf_reg': 0.18827111301413596, 'min_child_samples': 10} because of the following error: KeyboardInterrupt('').
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_87/1766362437.py", line 58, in objective
    model.fit(X_train,
  File "/usr/local/lib/python3.11/dist-packages/catboost/core.py", line 5873, in fit
    return self._fit(X, y, cat_features, text_features, embedding_features, None, graph, sample_weight, None, None, None, None, baseline,
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/catboost/core.py", line 2410, in _fit
    self

KeyboardInterrupt: 

In [ ]:
if use_optuna:
    trial = study.best_trial
    print('-'*70)
    print('- Optimize Result')
    print("- Best Trial OOF: {}".format(trial.value))
    print('-'*70)

    print('Number of finished trials:', len(study.trials))
    print('Best trial:', study.best_trial.params)
    
    print("Best trial Params:")
    trial = study.best_trial
    print('-'*70)
    print("--Best Trial OOF: {}".format(trial.value))
    print("--Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

# **Train**

In [ ]:
# pr = joblib.load('model/TREE_EXP-20/cat_params.pkl')

In [ ]:
# prA

In [ ]:
def make_model(seed, SAVE_PATH):

    
    '''
    Parameters
    '''
    lgb_model_params =  {'seed':42,
                         "boosting_type": "gbdt",
                         "metric": "rmse",
                         'objective':'regression',  # 回帰用
                         'n_estimators': 100000,
                         'learning_rate': 0.006194260637684729, 'max_depth': 311, 'num_leaves': 268, 'min_data_in_leaf': 94, 'min_child_samples': 20, 'feature_fraction': 0.9341247061167397, 'bagging_fraction': 0.9193456524233429, 'bagging_freq': 173, 'lambda_l1': 0.05514221196922455, 'lambda_l2': 0.0069745121796346, 'max_bin': 149, 'reg_alpha': 0.00012895825647239357, 'reg_lambda': 0.00853660379150604,
                         'verbosity': -1,
                         'extra_trees': True,
                         'device_type': 'gpu'
                        }
    
    

    xgb_model_params = {'objective': 'reg:squarederror',  # 回帰用
                        'eval_metric': 'rmse',  # 回帰評価指標
                        'random_seed': 806,  # 事前に定義されたシード値
                        'n_estimators': 100000, # iteration数
                        'alpha': 0.21712823216787464, 'lambda': 0.002622815685268001, 'learning_rate': 0.001549527535857199, 'max_depth': 297, 'num_leaves': 263, 'gamma': 49.8648869860361}
    
    cb_cox_params = {'loss_function': 'Cox', 'grow_policy': 'SymmetricTree',
                     'n_estimators': 800, 'learning_rate': 0.092, 'l2_leaf_reg': 2.5,
                     'max_depth': 7, 'colsample_bylevel': 0.84, 'subsample': 0.9, 
                     'random_strength': 0.8, 'verbose': 400,}

    
    # joblib.dump(lgb_model_params, f'{SAVE_PATH}/lgb_params.pkl')
    joblib.dump(cat_model_params, f'{SAVE_PATH}/cat_params.pkl')
    # joblib.dump(xgb_model_params, f'{SAVE_PATH}/xgb_params.pkl')

    '''
    Models
    '''
    models = {}
    
    # LightGBMの回帰モデル
    lgb_model = LGBMRegressor(**lgb_model_params, categorical_feature=categorical_cols)
    
    # CatBoostの回帰モデル
    cat_model = CatBoostRegressor(**cat_model_params, 
                                  cat_features=categorical_cols)
    
    # XGBoostの回帰モデル
    xgb_model = XGBRegressor(**xgb_model_params, 
                             enable_categorical=True,
                             tree_method='gpu_hist')
    
    # models['lgb_model'] = lgb_model
    models['cat_model'] = cat_model
    # models['xgb_model'] = xgb_model

    return models


In [ ]:
###################
# 
###################


EXP_NUM = 'EXP-1'
SAVE_PATH = f'notebook/output/TREE_{EXP_NUM}'

if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)

models = make_model(seed=my_config.general.seed, 
                    SAVE_PATH=SAVE_PATH)

oof_preds = {}
all_true = []

TRAIN_FLAG = True
fold_imp_flag = False
if TRAIN_FLAG:
    for model_name in models:
        oof_preds[model_name] = np.zeros((len(data), 1))

    for fold in range(my_config.general.n_splits):
        
        print('')
        print(clr.GREEN+'#'*25+clr.END)
        print(clr.GREEN+f'### Fold {fold+1}'+clr.END)
        print(clr.GREEN+'#'*25+clr.END)
        print('')
        
        
        '''
        Datasets
        '''
        
        if fold_imp_flag:    
            data = FE.drop_cols_fi2(df=data, 
                                    save_path='model',
                                    exp_num='TREE_EXP-16',
                                    nfold=fold,
                                    topn=400)
            FEATURES = data.drop(TARGETS+['fold'], axis=1).columns.tolist()
            
        X_train = data[data["fold"] != fold][FEATURES]
        y_train = data[data["fold"] != fold]['utility_agent1']
        X_val = data[data["fold"] == fold][FEATURES]
        y_val = data[data["fold"] == fold]['utility_agent1']
        valid_index = X_val.index
        for model_name in models:
            print(clr.RED+'*'*10+f' {model_name} '+'*'*10+clr.END)
            model = models[model_name]

            '''
            lightgbm
            '''
            if model_name.split('_')[0] == 'lgb':
                early_stopping_callback = lgb.early_stopping(400, first_metric_only=True, verbose=False)
                verbose_callback = lgb.log_evaluation(300)

                model.fit(X_train, y_train,
                          eval_metric='rmse',
                          eval_set=[(X_val, y_val)], 
                          callbacks=[early_stopping_callback, verbose_callback])

                oof_preds[model_name][valid_index] = model.predict(X_val).reshape(-1, 1)
                model.booster_.save_model(f'{SAVE_PATH}/LGB_{EXP_NUM}_f{fold}.txt')

            '''
            catboost
            '''
            if model_name.split('_')[0] == 'cat':
                model.fit(X_train,
                          y_train,
                          eval_set=[(X_val, y_val)],
                          verbose=400, # log300毎に表示
                          early_stopping_rounds=400)
                
                oof_preds[model_name][valid_index] = model.predict(X_val).reshape(-1, 1)
                joblib.dump(model, f'{SAVE_PATH}/CAT_{EXP_NUM}_f{fold}.pkl')

            '''
            xgboost
            '''
            if model_name.split('_')[0] == 'xgb':
                eval_set = [(X_train, y_train), (X_val, y_val)]
                model.fit(X_train, y_train, 
                          eval_set=eval_set, 
                          early_stopping_rounds=400,
                          verbose=400)
                oof_preds[model_name][valid_index] = model.predict(X_val).reshape(-1, 1)
                joblib.dump(model, f'{SAVE_PATH}/XGB_{EXP_NUM}_f{fold}.pkl')
            del X_train, X_val, y_train, y_val, model
            gc.collect()

    joblib.dump(oof_preds, f'{SAVE_PATH}/oof_preds_{EXP_NUM}.pkl')
    

# **Calculate CV**

In [ ]:
EXP_NUM = 'EXP-48'
SAVE_PATH = f'{my_config.general.model_save_path}/TREE_{EXP_NUM}'
shutil.copy('config.py', SAVE_PATH)

oof_preds = joblib.load(f'{SAVE_PATH}/oof_preds_{EXP_NUM}.pkl')
all_true = train['utility_agent1']
# all_true.columns = range(all_true.shape[1])

In [ ]:
for model_name in oof_preds:
    oof = pd.DataFrame(oof_preds[f'{model_name}'].copy(), columns=['pred'])
    oof['id'] = np.arange(len(oof))

    true = pd.DataFrame(all_true.copy())
    true['id'] = np.arange(len(true))
    
    cv = RMSE(true['utility_agent1'], oof['pred'])
    print(clr.RED + f'RMSE for {model_name} = {cv}' + clr.END)

# **DEBUG**

In [ ]:
EXP_NUM = 'EXP-19'
SAVE_PATH = f'{my_config.general.model_save_path}/TREE_{EXP_NUM}'


i=0
model = joblib.load(f'{SAVE_PATH}/CAT_{EXP_NUM}_f{i}.pkl')
cat_feature_indices = model.get_cat_feature_indices()
cat_columns = data[FEATURES].columns[cat_feature_indices]

In [ ]:
cat_columns

In [ ]:
column_name = 'p1_selection'
column_index = data[FEATURES].columns.get_loc(column_name)
column_index

In [ ]:
cat_feature_indices